---
# this is YAML front matter
layout: post
title: Python Websocket Quiz
description: Python can also be used to serve a websocket, which is like a miniature server that you can connect to and perform input/output.
courses: { compsci: { week: 3 }}
categories: [C4.0]
type: hacks
---

In [ ]:
import asyncio
import getpass
import sys
import websockets
import nest_asyncio

nest_asyncio.apply() # Required to make asyncio work in Jupyter Notebook

async def question_with_response(prompt, websocket): # Function used for each question
    await websocket.send("Question: " + prompt)
    msg = await websocket.recv()
    return msg

async def quiz(websocket, path): # Main quiz function
    questions = 5
    correct = 0

    await websocket.send('Hello, ' + getpass.getuser() + " running " + sys.executable)
    await websocket.send("You will be asked " + str(questions) + " questions.")
    await websocket.send("Question: " + "Are you ready to take a test?")
    rsp = await websocket.recv()
    if rsp == "no":
        await websocket.send("Alright then, later I guess?")
        await websocket.close()
    else:
        await websocket.send("Let's go!")
    rsp = await question_with_response("What command is used to include other functions that were previously developed?", websocket)
    if rsp == "import":
        await websocket.send(rsp + " is correct!")
        correct += 1
    else:
        await websocket.send(rsp + " is incorrect!")

    rsp = await question_with_response("What command is used to evaluate correct or incorrect response in this example?", websocket)
    if rsp == "if":
        await websocket.send(rsp + " is correct!")
        correct += 1
    else:
        await websocket.send(rsp + " is incorrect!")

    rsp = await question_with_response("Each 'if' command contains an '_________' to determine a true or false condition?", websocket)
    if rsp == "expression":
        await websocket.send(rsp + " is correct!")
        correct += 1
    else:
        await websocket.send(rsp + " is incorrect!")

    rsp = await question_with_response("Which command is the opposite of if?", websocket)
    if rsp == "else":
        await websocket.send(rsp + " is correct!")
        correct += 1
    else:
        await websocket.send(rsp + " is not correct!")

    rsp = await question_with_response("What are we using to run the ipynb files?", websocket)
    if rsp == "Jupyter Notebook":
        await websocket.send(rsp + " is correct!")
        correct += 1
    else:
        await websocket.send(rsp +  " is not correct!")

    score = (correct * 100) / questions # Percentage calculation

    await websocket.send(str(score) + "% This is your percentage")
    await websocket.send(getpass.getuser() + " you scored " + str(correct) + "/" + str(questions))

    await websocket.close()

start_server = websockets.serve(quiz, "localhost", 8765)

async def main(): # Function to start the websocket
    await start_server

if __name__ == "__main__": # Actually start the websocket
    asyncio.run(main())


### Brief Code Explanation
